Author: Roel De Los Santos <br />
Topic: Module 5 Challenge <br />
Date: 8/10/23

# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file city_data_df = pd.read_csv("cities.csv")
cities_df = pd.read_csv("cities.csv")
# Display 
cities_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,amga,60.894218,131.979980,17.84,70,99,1.95,RU,1691644042
1,edinburgh of the seven seas,-37.067589,-12.310830,10.33,59,67,10.09,SH,1691644043
2,san patricio,27.969532,-97.773172,29.36,81,0,7.76,US,1691644044
3,puerto ayora,-0.747167,-90.313420,24.55,94,61,2.68,EC,1691644045
4,bowie,38.942446,-76.730172,24.90,87,0,2.57,US,1691644046


---

### Step 1: Create a map that displays a point for every city called "cities_df'

In [3]:
# Configure the map plot
coordinates = {
    "Lat": (cities_df["Lat"]),
    "Lng": (cities_df["Lng"]),
    "City": (cities_df["City"]),
    "Humidity": (cities_df["Humidity"])
}

coordinates_df = pd.DataFrame(coordinates)

# Display the map

humidity_map = coordinates_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    color = "City"
)
humidity_map

c:\Users\roelj\anaconda3\anaconda\envs\dev2\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: use 'cities_df` to find your ideal weather condition

In [4]:
# cities that fit criteria and null values
ideal_weather = cities_df.loc[(cities_df["Max Temp"] >= 10) & 
                            (cities_df["Max Temp"] < 30).dropna()]
# Display 
display(ideal_weather)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,amga,60.894218,131.979980,17.84,70,99,1.95,RU,1691644042
1,edinburgh of the seven seas,-37.067589,-12.310830,10.33,59,67,10.09,SH,1691644043
2,san patricio,27.969532,-97.773172,29.36,81,0,7.76,US,1691644044
3,puerto ayora,-0.747167,-90.313420,24.55,94,61,2.68,EC,1691644045
4,bowie,38.942446,-76.730172,24.90,87,0,2.57,US,1691644046
...,...,...,...,...,...,...,...,...,...
553,mogadishu,2.034931,45.341918,25.13,81,91,7.41,SO,1691644578
554,yeraliyev,48.647499,70.372314,25.64,45,97,3.23,KZ,1691644579
555,paita,-5.085164,-81.113679,21.47,83,5,3.95,PE,1691644580
556,panacan,7.151479,125.658660,29.64,71,99,2.36,PH,1691644581


### Step 3: Create a new DataFrame called `htl_df`.

In [5]:
# create DataFrame called hotel_df for the city, country, coordinates, and humidity
htl_df = ideal_weather[["City", "Country", "Lat", "Lng", "Humidity"]].copy()


# Add an empty column, "Hotel Name," 
htl_df["Hotel Name"] = " "

# Display 
htl_df

,City,Country,Lat,Lng,Humidity,Hotel Name
0,amga,RU,60.894218,131.979980,70,
1,edinburgh of the seven seas,SH,-37.067589,-12.310830,59,
2,san patricio,US,27.969532,-97.773172,81,
3,puerto ayora,EC,-0.747167,-90.313420,94,
4,bowie,US,38.942446,-76.730172,87,
...,...,...,...,...,...,...
553,mogadishu,SO,2.034931,45.341918,81,
554,yeraliyev,KZ,48.647499,70.372314,45,
555,paita,PE,-5.085164,-81.113679,83,
556,panacan,PH,7.151479,125.658660,71,


### Step 4: For each city located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
categories = ["accommodation.hotel"]
params = {"categories": categories,
"apiKey" : geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in htl_df.iterrows():
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params = params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        htl_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        htl_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{htl_df.loc[index, 'City']} - nearest hotel: {htl_df.loc[index, 'Hotel Name']}")

# Display sample data
htl_df

Starting hotel search
amga - nearest hotel: No hotel found
edinburgh of the seven seas - nearest hotel: No hotel found
san patricio - nearest hotel: No hotel found
puerto ayora - nearest hotel: No hotel found
bowie - nearest hotel: No hotel found
adamstown - nearest hotel: No hotel found
puerto armuelles - nearest hotel: No hotel found
fort bragg - nearest hotel: No hotel found
okha - nearest hotel: No hotel found
zaragoza - nearest hotel: No hotel found
puerto san carlos - nearest hotel: No hotel found
pangody - nearest hotel: No hotel found
vilyuchinsk - nearest hotel: No hotel found
lebu - nearest hotel: No hotel found
waitangi - nearest hotel: No hotel found
lobito - nearest hotel: No hotel found
tura - nearest hotel: No hotel found
kodiak - nearest hotel: No hotel found
gadzhiyevo - nearest hotel: No hotel found
colonia - nearest hotel: No hotel found
talnakh - nearest hotel: No hotel found
bargara - nearest hotel: No hotel found
natal - nearest hotel: No hotel found
fortuna - nea

,City,Country,Lat,Lng,Humidity,Hotel Name
0,amga,RU,60.894218,131.979980,70,No hotel found
1,edinburgh of the seven seas,SH,-37.067589,-12.310830,59,No hotel found
2,san patricio,US,27.969532,-97.773172,81,No hotel found
3,puerto ayora,EC,-0.747167,-90.313420,94,No hotel found
4,bowie,US,38.942446,-76.730172,87,No hotel found
...,...,...,...,...,...,...
553,mogadishu,SO,2.034931,45.341918,81,No hotel found
554,yeraliyev,KZ,48.647499,70.372314,45,No hotel found
555,paita,PE,-5.085164,-81.113679,83,No hotel found
556,panacan,PH,7.151479,125.658660,71,No hotel found


### Step 5: Add the hotel name and the country for cities in the map.

In [7]:
# Configure the map plot
htl_map = htl_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    color = "City",
    hover_cols = ["Hotel Name", "Country"]
)
# Display the map
htl_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)